In [ ]:
'''
參考連結:
[1] Implement Okapi BM25 variants in Gensim #3304
https://github.com/piskvorky/gensim/pull/3304
[2] 年報審閱分工 #2
https://github.com/GufoLAB/annual_report_verification/issues/2
[3] 法規名稱： 銀行年報應行記載事項準則 (修正日期：	民國 113 年 02 月 19 日)
https://law.moj.gov.tw/LawClass/LawAll.aspx?PCODE=G0380104
[4] 法規名稱： 公開發行公司年報應行記載事項準則 (修正日期：	民國 112 年 11 月 10 日)
https://law.moj.gov.tw/LawClass/LawAll.aspx?pcode=G0400022
[5] gensim - models.tfidfmodel
https://radimrehurek.com/gensim/models/tfidfmodel.html
[6] SMART Information Retrieval System
https://en.wikipedia.org/wiki/SMART_Information_Retrieval_System
'''

In [1]:
import re
import sqlite3
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
import jieba
import heapq

In [30]:
# 資料庫連線
db_file = '2633.db' # 2303 2633
conn = sqlite3.connect(db_file)
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [31]:
# 分詞設定
jieba.load_userdict("userdict.txt")

In [32]:
# 匯出所有資料
sql = f'''
SELECT *
FROM `pdf`
ORDER BY `id` ASC
'''
stmt = cursor.execute(sql)
list_data = []
list_page_num = []
for row in stmt.fetchall():
    text_content = row['text_content']
    table_format = row['table_format']
    # if table_format == None: 
    #     list_data.append(text_content)
    # else:
    #     list_data.append(text_content +  table_format)
    list_data.append(text_content)
    list_page_num.append(row['page_num'])

In [33]:
# 關閉 sqlite
cursor.close()
conn.close()

In [34]:
# 斷詞
list_seq = []
for seq in list_data:
    seq = re.sub(r"\s| |\n", "", seq) # 簡單清理
    list_seq.append(list(jieba.cut(seq, HMM=False)))

In [35]:
# 用 BM25 模型計算相似度(相關性)
def bm25_similarity(list_seq, query):
    dictionary = Dictionary(list_seq)
    bm25_model = OkapiBM25Model(dictionary=dictionary)
    bm25_corpus = bm25_model[list(map(dictionary.doc2bow, list_seq))]
    bm25_index = SparseMatrixSimilarity(bm25_corpus, num_docs=len(list_seq), num_terms=len(dictionary), normalize_queries=False, normalize_documents=False)
    query = list(jieba.cut(query))
    tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn') # default: nfc
    tfidf_query = tfidf_model[dictionary.doc2bow(query)]
    similarities = bm25_index[tfidf_query]
    return similarities

In [36]:
# 顯示前 top_k 高相似度的文章
def get_page_score(similarities, top_k):
    similarities = list(similarities)
    top_values = heapq.nlargest(top_k, similarities)
    top_indices = [similarities.index(value) for value in top_values]
    return [(list_page_num[doc_index], top_values[index]) for index, doc_index in enumerate(top_indices)]

In [37]:
# 基本設定
top_k = 5

In [ ]:
'''
第一類
18.4	(四)營運概況 4.環保支出資訊：最近年度及截至年報刊印日止，因污染環境所遭受之損失（包括賠償及環境保護稽查結果違反環保法規事項，應列明處分日期、處分字號、違反法規條文、違反法規內容、處分內容），並揭露目前及未來可能發生之估計金額與因應措施，如無法合理估計者，應說明其無法合理估計之事實。
18.5.1	5.勞資關係 (1)列示公司各項員工福利措施、進修、訓練、退休制度與其實施情形，以及勞資間之協議與各項員工權益維護措施情形。
18.5.2	(2)列明最近年度及截至年報刊印日止，因勞資糾紛所遭受之損失（包括勞工檢查結果違反勞動基準法事項，應列明處分日期、處分字號、違反法規條文、違反法規內容、處分內容），並揭露目前及未來可能發生之估計金額與因應措施，如無法合理估計者，應說明其無法合理估計之事實。

第二類
11.1	3.公司資本及股份 (1)股本來源是否按附表五揭露。
11.2	(2)股東結構是否按附表六揭露。
11.3	(3)普通股及特別股之股權分散情形是否按附表七揭露。
11.4	(4)主要股東名單是否按附表八揭露，應列明股權比例達百分之五以上之股東，如不足十名，應揭露至股權比例占前十名之股東名稱、持股數額及比例。
11.5	(5)最近二年度每股資料是否按附表九揭露。
11.9.1	(9)公司買回本公司股份情形 A.已執行完畢者：是否按附表十(1)揭露。
11.9.2	B.尚在執行中者：是否按附表十(2)揭露。

第四類
18.1.1	(四)營運概況 1.業務內容 (1)業務範圍。
18.1.2	(2)產業概況。
18.1.3	(3)技術及研發概況。
18.1.4	(4)長、短期業務發展計畫。
'''

In [38]:
# 第 1 類: 18.4
query = '''
(四)營運概況 4.環保支出資訊：最近年度及截至年報刊印日止，因污染環境所遭受之損失（包括賠償及環境保護稽查結果違反環保法規事項，應列明處分日期、處分字號、違反法規條文、違反法規內容、處分內容），並揭露目前及未來可能發生之估計金額與因應措施，如無法合理估計者，應說明其無法合理估計之事實。
'''
query = '''
營運概況 環保支出資訊 污染環境
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 177, Score: 15.5245
Page Num: 178, Score: 9.0146
Page Num: 117, Score: 7.6217
Page Num: 101, Score: 6.0963
Page Num: 160, Score: 4.8552


In [39]:
# 第 1 類: 18.5.1
query = '''
5.勞資關係 (1)列示公司各項員工福利措施、進修、訓練、退休制度與其實施情形，以及勞資間之協議與各項員工權益維護措施情形。
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 62, Score: 52.6264
Page Num: 60, Score: 50.6946
Page Num: 104, Score: 39.3249
Page Num: 107, Score: 36.1723
Page Num: 106, Score: 35.1926


In [40]:
# 第 1 類: 18.5.2
query = '''
(2)列明最近年度及截至年報刊印日止，因勞資糾紛所遭受之損失（包括勞工檢查結果違反勞動基準法事項，應列明處分日期、處分字號、違反法規條文、違反法規內容、處分內容），並揭露目前及未來可能發生之估計金額與因應措施，如無法合理估計者，應說明其無法合理估計之事實。
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 63, Score: 153.2087
Page Num: 169, Score: 77.7168
Page Num: 177, Score: 65.9090
Page Num: 123, Score: 65.7830
Page Num: 265, Score: 61.5669


In [41]:
# 第 2 類: 11.1
query = '''
3.公司資本及股份 (1)股本來源是否按附表五揭露。
'''
query = '''
公司資本及股份 股本來源 發行價格 核定股本 實收股本
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 133, Score: 24.1517
Page Num: 132, Score: 16.5357
Page Num: 268, Score: 11.8168
Page Num: 229, Score: 9.1628
Page Num: 230, Score: 7.7453


In [42]:
# 第 2 類: 11.2
query = '''
(2)股東結構是否按附表六揭露。
'''
query = '''
股東結構 政府機構 金融機構 其他法人
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 133, Score: 19.5871
Page Num: 238, Score: 10.6320
Page Num: 39, Score: 10.0415
Page Num: 38, Score: 9.7473
Page Num: 93, Score: 9.5165


In [43]:
# 第 2 類: 11.3
query = '''
(3)普通股及特別股之股權分散情形是否按附表七揭露。
'''
query = '''
普通股 特別股 股權分散情形 持股分級 股東人數 持有股數 持股比例
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 134, Score: 33.1135
Page Num: 248, Score: 27.9088
Page Num: 129, Score: 19.8325
Page Num: 130, Score: 19.0410
Page Num: 133, Score: 18.3731


In [44]:
# 第 2 類: 11.4
query = '''
(4)主要股東名單是否按附表八揭露，應列明股權比例達百分之五以上之股東，如不足十名，應揭露至股權比例占前十名之股東名稱、持股數額及比例。
'''
query = '''
主要股東名單 主要股東名稱 持有股數 持股比例
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 134, Score: 26.4068
Page Num: 248, Score: 22.8031
Page Num: 129, Score: 22.7466
Page Num: 130, Score: 22.5653
Page Num: 128, Score: 14.6258


In [45]:
# 第 2 類: 11.5
query = '''
(5)最近二年度每股資料是否按附表九揭露。
'''
query = '''
最近二年度每股資料
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 135, Score: 12.8242
Page Num: 268, Score: 8.8491
Page Num: 235, Score: 7.8716
Page Num: 134, Score: 7.0798
Page Num: 229, Score: 6.2786


In [46]:
# 第 2 類: 11.9.1
query = '''
(9)公司買回本公司股份情形 A.已執行完畢者：是否按附表十(1)揭露。
'''
query = '''
公司買回本公司股份情形 已執行完畢者 買回期次 買回目的
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 136, Score: 5.2576
Page Num: 182, Score: 3.3433
Page Num: 149, Score: 3.2121
Page Num: 10, Score: 3.1465
Page Num: 211, Score: 3.0266


In [47]:
# 第 2 類: 11.9.2
query = '''
B.尚在執行中者：是否按附表十(2)揭露。
'''
query = '''
尚在執行中者 買回期次 買回目的 買回股份之種類 買回股份之總金額上限
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 138, Score: 9.5838
Page Num: 139, Score: 9.1948
Page Num: 137, Score: 8.9273
Page Num: 133, Score: 8.5844
Page Num: 202, Score: 7.6316


In [48]:
# 第 4 類: 18.1.1
query = '''
(四)營運概況 1.業務內容 (1)業務範圍。
'''
query = '''
營運概況 業務內容 業務範圍
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 142, Score: 13.3891
Page Num: 4, Score: 9.5812
Page Num: 141, Score: 4.7363
Page Num: 150, Score: 4.3252
Page Num: 152, Score: 3.5155


In [49]:
# 第 4 類: 18.1.2
query = '''
(2)產業概況。
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 152, Score: 14.2275
Page Num: 40, Score: 10.1357
Page Num: 41, Score: 10.1302
Page Num: 83, Score: 10.0836
Page Num: 198, Score: 10.0579


In [50]:
# 第 4 類: 18.1.3
query = '''
(3)技術及研發概況。
'''
query = '''
技術及研發概況
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 5, Score: 6.4657
Page Num: 163, Score: 4.7365
Page Num: 1, Score: 0.0000
Page Num: 1, Score: 0.0000
Page Num: 1, Score: 0.0000


In [51]:
# 第 4 類: 18.1.4
query = '''
(4)長、短期業務發展計畫。
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

Page Num: 158, Score: 16.5780
Page Num: 167, Score: 16.2447
Page Num: 155, Score: 14.4646
Page Num: 165, Score: 14.0266
Page Num: 222, Score: 13.6905
